In [ ]:
import numpy as np
import xarray as xr

In [ ]:
def add_s_layers(river_ds):
    # Creating new array to fill the gaps
    v_data = np.zeros((5, 175))
    v_add = xr.DataArray(v_data, dims=river_ds['river_Vshape'].dims, coords=river_ds['river_Vshape'].coords)
    temp_data = np.ones((4019, 5, 175)) * 5
    t_add = xr.DataArray(temp_data, dims=river_ds['river_temp'].dims, coords=river_ds['river_temp'].coords)
    salt_data = np.ones((4019, 5, 175))
    s_add = xr.DataArray(salt_data, dims=river_ds['river_salt'].dims, coords=river_ds['river_salt'].coords)
    
    # New ready data arrays
    river_Vshape = xr.concat([v_add, river_ds['river_Vshape']], dim="s_rho")
    river_temp = xr.concat([t_add, river_ds['river_temp']], dim="s_rho")
    river_salt = xr.concat([s_add, river_ds['river_salt']], dim="s_rho")
    
    # Remove old and add new variables with new s_rho=40 dimension
    river_ds = river_ds.drop_dims('s_rho')
    river_ds = river_ds.assign(river_Vshape=river_Vshape, river_temp=river_temp, river_salt=river_salt)

    return river_ds

In [ ]:
ds = xr.open_dataset('fram_data/norfjords_160m_river_new.nc')

In [ ]:
ds.river_transport.isel(river_time=0, river=0)

In [ ]:
# ds['river_transport'].values = np.zeros((4019, 175))

In [ ]:
for i in range(ds.dims['river']):
    river = ds.isel(river=i, river_time=0)
    cell_runoff = (abs(river.river_transport) * river.river_Vshape).max().values
    if cell_runoff > 1:
        print(f"River {i+1} max runoff: {cell_runoff}")
        print(f"Coordinates: {river.river_Xposition}; {river.river_Eposition} \n")

In [ ]:
# ds.to_netcdf(path='fram_data/norfjords_160m_river_zero_runoff.nc', format='NETCDF4')